In [169]:
import requests
import pandas as pd
import json


In [64]:
#Conseguir la lista de parlamentarios con sus ID
parlamentarios = requests.get('http://datos.congreso.gov.py/opendata/api/data/parlamentario').text

In [42]:
# Cargar el JSON a un DataFrame
df = pd.read_json(parlamentarios)
# Guardar el DF como un archivo CSV 
df.to_csv("parlamentarios.csv", index=False)

In [65]:
parlamentarios = json.loads(parlamentarios)

In [179]:
# Conseguir el dataset de parlamentarios.
parlamentarios = requests.get('http://datos.congreso.gov.py/opendata/api/data/parlamentario').text

# Cargar el JSON en un Data Frame para leer con pandas.
df_parlamentarios = pd.read_json(parlamentarios)

# Conseguir la lista de ids del data frame.
congressman_ids = df_parlamentarios['idParlamentario'].tolist()

In [183]:
# Crear una lista vacia de dataframes.
dataframes = []

In [ ]:

# Definir una funcion que extrae dos atributos de la columna proyectos
def extract_proyecto(row):
 proyecto = row['proyectos']
 # Chequear si el campo proyecto esta vacio
 if pd.isna(proyecto) or len(proyecto) == 0:
  # Regresar dos strings vacios con el indice dado
  return pd.Series(['', ''], index=['idProyecto', 'iniciativa'])
 else:
  # Proceder como antes
  idProyecto = proyecto.get('idProyecto', '')
  iniciativa = proyecto.get('iniciativa', '')
  return pd.Series([idProyecto, iniciativa], index=['idProyecto', 'iniciativa'])

In [ ]:
# Loop a traves de la lista y traer la informacion de cada congresista.
for congressman_id in congressman_ids:
 # Intentar hacer el pedido
 try:
  # Se hace el pedido con un timeout a 60 segundos por la cantidad de pedidos grandes
  datos_congresistas = requests.get(f'http://datos.congreso.gov.py/opendata/api/data/parlamentario/{congressman_id}/proyectos', timeout=60).text
 except requests.exceptions.ConnectionError as e:
  # Manejar los errores de coneccion
  print(f'Connection error for id {congressman_id}: {e}')
  # Saltar esta iteracion
  continue
 except requests.exceptions.ReadTimeout as e:
  # Manejar los errores de timeout
  print(f'Read timeout error for id {congressman_id}: {e}')
  # Saltar esta iteracion
  continue

 # Pasar el JSON a un objeto de Python
 datos_congresistas = json.loads(datos_congresistas)

 # Chequear si datos_congresistas esta vacio
 if not datos_abdo: # Va a retornar falso si datos_congresistas es un objeto vacio
  # Saltar esta iteracion
  continue
 else:
  # Debido a otros errores, debemos revisar si datos_congresistas posee valores escalares
  if type(datos_congresistas) == dict and all(type(v) in (int, float, str, bool) for v in datos_congresistas.values()):
   # Saltar esta iteracion
   continue
  else:
   # Cargar el objeto de Python a Data Frames.
   df = pd.DataFrame(datos_congresistas)

   # Aplicar la funcion para extraer datos de la columna proyectos, en particular el ide del proyecto y la iniciativa.
   df[['idProyecto', 'iniciativa']] = df.apply(extract_proyecto, axis=1)

   # Remover la palabra 'PARLAMENTARIA:' al inicio de los nombres de los autores del proyecto de ley.
   df['iniciativa'] = df['iniciativa'].str.replace('PARLAMENTARIA:', '')

   # Agregar el dataframes a la lista.
   dataframes.append(df)

In [ ]:
# Combinar los dos dataframes.
df = pd.concat(dataframes)

In [ ]:
# Eliminar o 'soltar' la columna proyectos para reducir el tamano final del documento.
df = df.drop('proyectos', axis=1)

In [ ]:

# Guardar el resultado en csv.
df.to_csv('congresistas_proyectos.csv', index=False)